# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 9 2022 10:03AM,246038,12,8505167,"SD Head USA, LLC",Released
1,Sep 9 2022 9:55AM,246037,19,60017713,"GUSA Granules USA, Inc.",Released
2,Sep 9 2022 9:53AM,246036,20,ALUR371453915,Alumier Labs Inc.,Released
3,Sep 9 2022 9:53AM,246036,20,ALUR871112990,Alumier Labs Inc.,Released
4,Sep 9 2022 9:53AM,246036,20,ALUR881582253,Alumier Labs Inc.,Released
5,Sep 9 2022 9:53AM,246036,20,ALUR993006149,Alumier Labs Inc.,Released
6,Sep 9 2022 9:52AM,246020,20,ALU0007869,Alumier Labs Inc.,Released
7,Sep 9 2022 9:48AM,246035,10,PRF-36633,Bio-PRF,Released
8,Sep 9 2022 9:48AM,246035,10,PRF-36645,Bio-PRF,Released
9,Sep 9 2022 9:48AM,246035,10,PRF-36652,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,246031,Released,16
34,246035,Released,9
35,246036,Released,4
36,246037,Released,1
37,246038,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246031,NaN,NaN,16.0
246035,NaN,NaN,9.0
246036,NaN,NaN,4.0
246037,NaN,NaN,1.0
246038,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245651,0.0,0.0,1.0
245733,0.0,1.0,0.0
245744,0.0,1.0,0.0
245763,0.0,0.0,1.0
245771,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245651,0,0,1
245733,0,1,0
245744,0,1,0
245763,0,0,1
245771,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245651,0,0,1
1,245733,0,1,0
2,245744,0,1,0
3,245763,0,0,1
4,245771,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245651,,,1
1,245733,,1,
2,245744,,1,
3,245763,,,1
4,245771,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC"
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc."
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.
6,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.
7,Sep 9 2022 9:48AM,246035,10,Bio-PRF
16,Sep 9 2022 9:45AM,246023,20,"ACI Healthcare USA, Inc."
44,Sep 9 2022 9:31AM,246031,15,"Alliance Pharma, Inc."
60,Sep 9 2022 9:25AM,246026,10,"Methapharm, Inc."
61,Sep 9 2022 9:23AM,246025,12,Hush Hush
62,Sep 9 2022 8:44AM,246022,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC",,,1
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc.",,,1
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.,,,4
3,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.,,,1
4,Sep 9 2022 9:48AM,246035,10,Bio-PRF,,,9
5,Sep 9 2022 9:45AM,246023,20,"ACI Healthcare USA, Inc.",,,28
6,Sep 9 2022 9:31AM,246031,15,"Alliance Pharma, Inc.",,,16
7,Sep 9 2022 9:25AM,246026,10,"Methapharm, Inc.",,,1
8,Sep 9 2022 9:23AM,246025,12,Hush Hush,,,1
9,Sep 9 2022 8:44AM,246022,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC",1,,
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc.",1,,
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.,4,,
3,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.,1,,
4,Sep 9 2022 9:48AM,246035,10,Bio-PRF,9,,
5,Sep 9 2022 9:45AM,246023,20,"ACI Healthcare USA, Inc.",28,,
6,Sep 9 2022 9:31AM,246031,15,"Alliance Pharma, Inc.",16,,
7,Sep 9 2022 9:25AM,246026,10,"Methapharm, Inc.",1,,
8,Sep 9 2022 9:23AM,246025,12,Hush Hush,1,,
9,Sep 9 2022 8:44AM,246022,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC",1,,
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc.",1,,
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.,4,,
3,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.,1,,
4,Sep 9 2022 9:48AM,246035,10,Bio-PRF,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC",1.0,NaN,NaN
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.,4.0,NaN,NaN
3,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.,1.0,NaN,NaN
4,Sep 9 2022 9:48AM,246035,10,Bio-PRF,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 10:03AM,246038,12,"SD Head USA, LLC",1.0,0.0,0.0
1,Sep 9 2022 9:55AM,246037,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Sep 9 2022 9:53AM,246036,20,Alumier Labs Inc.,4.0,0.0,0.0
3,Sep 9 2022 9:52AM,246020,20,Alumier Labs Inc.,1.0,0.0,0.0
4,Sep 9 2022 9:48AM,246035,10,Bio-PRF,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1968086,31.0,9.0,0.0
12,492063,2.0,0.0,0.0
15,983949,42.0,12.0,75.0
16,737978,7.0,2.0,0.0
19,246037,1.0,0.0,0.0
20,984082,36.0,3.0,7.0
21,1720443,4.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1968086,31.0,9.0,0.0
1,12,492063,2.0,0.0,0.0
2,15,983949,42.0,12.0,75.0
3,16,737978,7.0,2.0,0.0
4,19,246037,1.0,0.0,0.0
5,20,984082,36.0,3.0,7.0
6,21,1720443,4.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,9.0,0.0
1,12,2.0,0.0,0.0
2,15,42.0,12.0,75.0
3,16,7.0,2.0,0.0
4,19,1.0,0.0,0.0
5,20,36.0,3.0,7.0
6,21,4.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,2.0
2,15,Released,42.0
3,16,Released,7.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,75.0,0.0,0.0,7.0,0.0
Executing,9.0,0.0,12.0,2.0,0.0,3.0,3.0
Released,31.0,2.0,42.0,7.0,1.0,36.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,75.0,0.0,0.0,7.0,0.0
1,Executing,9.0,0.0,12.0,2.0,0.0,3.0,3.0
2,Released,31.0,2.0,42.0,7.0,1.0,36.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,75.0,0.0,0.0,7.0,0.0
1,Executing,9.0,0.0,12.0,2.0,0.0,3.0,3.0
2,Released,31.0,2.0,42.0,7.0,1.0,36.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()